## 1. Introduction
<p><a href="https://commons.wikimedia.org/wiki/File:Tux.svg">
<img style="float: right;margin:5px 20px 5px 1px" width="150px" src="https://assets.datacamp.com/production/project_111/img/tux.png" alt="Tux - the Linux mascot">
</a></p>
<p>Version control repositories like CVS, Subversion or Git can be a real gold mine for software developers. They contain every change to the source code including the date (the "when"), the responsible developer (the "who"), as well as a little message that describes the intention (the "what") of a change.</p>
<p>In this notebook, we will analyze the evolution of a very famous open-source project &ndash; the Linux kernel. The Linux kernel is the heart of some Linux distributions like Debian, Ubuntu or CentOS. Our dataset at hand contains the history of kernel development of almost 13 years (early 2005 - late 2017). We get some insights into the work of the development efforts by </p>
<ul>
<li>identifying the TOP 10 contributors and</li>
<li>visualizing the commits over the years.</li>
</ul>

In [457]:
# Printing the content of git_log_excerpt.csv
import pandas as pd
df = pd.read_csv('datasets/git_log_excerpt.csv',sep='#',header=None)
display(df.head())

0               1
0  1502382966  Linus Torvalds
1  1501368308    Max Gurtovoy
2  1501625560     James Smart
3  1501625559     James Smart
4  1500568442    Martin Wilck

## 2. Reading in the dataset
<p>The dataset was created by using the command <code>git log --encoding=latin-1 --pretty="%at#%aN"</code> in late 2017. The <code>latin-1</code> encoded text output was saved in a header-less CSV file. In this file, each row is a commit entry with the following information:</p>
<ul>
<li><code>timestamp</code>: the time of the commit as a UNIX timestamp in seconds since 1970-01-01 00:00:00 (Git log placeholder "<code>%at</code>")</li>
<li><code>author</code>: the name of the author that performed the commit (Git log placeholder "<code>%aN</code>")</li>
</ul>
<p>The columns are separated by the number sign <code>#</code>. The complete dataset is in the <code>datasets/</code> directory. It is a <code>gz</code>-compressed csv file named <code>git_log.gz</code>.</p>

In [459]:
# Reading in the log file
git_log = pd.read_csv('datasets/git_log.gz',encoding='latin-1', sep='#',header=None, names=['timestamp','author'])

# Printing out the first 5 rows
display(git_log)

timestamp                    author
0       1502826583            Linus Torvalds
1       1501749089             Adrian Hunter
2       1501749088             Adrian Hunter
3       1501882480                 Kees Cook
4       1497271395                 Rob Clark
5       1501749087             Adrian Hunter
6       1501749086             Adrian Hunter
7       1502109470              Anton Volkov
8       1501571512               Keith Busch
9       1502807695              Alex Deucher
10      1497818483          Vijendar Mukunda
11      1499440633          Vijendar Mukunda
12      1502352765                 Evan Quan
13      1502349476                 Evan Quan
14      1502335483             Hawking Zhang
15      1502274646             Dan Carpenter
16      1502166030                   Rex Zhu
17      1501831897                   Rex Zhu
18      1502188971             Trigger Huang
19      1502193046              Kent Russell
20      1502193481              Kent Russell
21      1502192932              Kent Russell
22      1502192881              Kent Russell
23      1502091149                   Rex Zhu
24      1501836998                 Huang Rui
25      1501761515           Cihangir Akturk
26      1501761496           Cihangir Akturk
27      1497480374              Alex Deucher
28      1501673224                   Rex Zhu
29      1501670264                   Rex Zhu
...            ...                       ...
699041  1113690257            Paul Mackerras
699042  1113690257            Paul Mackerras
699043  1113690256            Paul Mackerras
699044  1113690255          Eugene Surovegin
699045  1113690254            Paul Mackerras
699046  1113690253              James Morris
699047  1113690251           Jean Tourrilhes
699048  1113690250                Herbert Xu
699049  1113690249  Arnaldo Carvalho de Melo
699050  1113690249           David S. Miller
699051  1113690248              Martin Hicks
699052  1113690247             akpm@osdl.org
699053  1113690246             akpm@osdl.org
699054  1113690245          Andrea Arcangeli
699055  1113690245                Jeff Moyer
699056  1113690244               Dave Airlie
699057  1113690243              James Morris
699058  1113690242             akpm@osdl.org
699059  1113690241           David S. Miller
699060  1113690241              Stas Sergeev
699061  1113690240             akpm@osdl.org
699062  1113690239           James Bottomley
699063  1113690238        Artem B. Bityuckiy
699064  1113690237             akpm@osdl.org
699065  1113690237             akpm@osdl.org
699066  1113690236             akpm@osdl.org
699067  1113690235             akpm@osdl.org
699068  1113690234                Neil Brown
699069  1113690233         Christoph Lameter
699070  1113690036            Linus Torvalds

[699071 rows x 2 columns]

## 3. Getting an overview
<p>The dataset contains the information about every single code contribution (a "commit") to the Linux kernel over the last 13 years. We'll first take a look at the number of authors and their commits to the repository.</p>

In [461]:
# calculating number of commits
number_of_commits = git_log.timestamp.count()

# calculating number of authors
number_of_authors = git_log.author.nunique()

# printing out the results
print("%s authors committed %s code changes." % (number_of_authors, number_of_commits))

17385 authors committed 699071 code changes.


## 4. Finding the TOP 10 contributors
<p>There are some very important people that changed the Linux kernel very often. To see if there are any bottlenecks, we take a look at the TOP 10 authors with the most commits.</p>

In [463]:
# Identifying the top 10 authors
top_10_authors = git_log.author.value_counts().head(10)

# Listing contents of 'top_10_authors'
top_10_authors

Linus Torvalds           23361
David S. Miller           9106
Mark Brown                6802
Takashi Iwai              6209
Al Viro                   6006
H Hartley Sweeten         5938
Ingo Molnar               5344
Mauro Carvalho Chehab     5204
Arnd Bergmann             4890
Greg Kroah-Hartman        4580
Name: author, dtype: int64

## 5. Wrangling the data
<p>For our analysis, we want to visualize the contributions over time. For this, we use the information in the <code>timestamp</code> column to create a time series-based column.</p>

In [465]:
# converting the timestamp column
git_log.timestamp=pd.to_datetime(git_log.timestamp,unit='s')

# summarizing the converted timestamp column
git_log.timestamp.describe()

count                  699071
unique                 668448
top       2008-09-04 05:30:19
freq                       99
first     1970-01-01 00:00:01
last      2037-04-25 08:08:26
Name: timestamp, dtype: object

## 6. Treating wrong timestamps
<p>As we can see from the results above, some contributors had their operating system's time incorrectly set when they committed to the repository. We'll clean up the <code>timestamp</code> column by dropping the rows with the incorrect timestamps.</p>

In [467]:
# determining the first real commit timestamp
first_commit_timestamp = git_log.timestamp.iloc[-1]

# determining the last sensible commit timestamp
last_commit_timestamp = last_commit_timestamp = pd.to_datetime('now')
# filtering out wrong timestamps
corrected_log = git_log[(git_log['timestamp']>=first_commit_timestamp) & (git_log['timestamp']<=last_commit_timestamp)]
# summarizing the corrected timestamp column
corrected_log.timestamp.describe()

count                  698569
unique                 667977
top       2008-09-04 05:30:19
freq                       99
first     2005-04-16 22:20:36
last      2017-10-03 12:57:00
Name: timestamp, dtype: object

## 7. Grouping commits per year
<p>To find out how the development activity has increased over time, we'll group the commits by year and count them up.</p>

In [469]:
# Counting the no. commits per year
commits_per_year = corrected_log.groupby(pd.Grouper(key='timestamp',freq='AS')).count()



## 8. Visualizing the history of Linux
<p>Finally, we'll make a plot out of these counts to better see how the development effort on Linux has increased over the the last few years. </p>

In [471]:
# Listing the first rows
commits_per_year.head()

# Setting up plotting in Jupyter notebooks
%matplotlib inline


# plot the data
ax=commits_per_year.plot(kind='bar',legend=None,title='Commits Per Year')
ax.set_xticklabels(commits_per_year.index.year)
ax.set_xlabel('Years')
ax.set_ylabel('Commits')

## 9.  Conclusion
<p>Thanks to the solid foundation and caretaking of Linux Torvalds, many other developers are now able to contribute to the Linux kernel as well. There is no decrease of development activity at sight!</p>

In [473]:
# calculating or setting the year with the most commits to Linux
year_with_most_commits = 2016